In [1]:
import torch
import pandas as pd
import os.path as osp
import numpy as np
from ast import literal_eval
from PIL import Image

In [2]:
## Prepare the  dataset (merge it with the emotion-histograms.)
datasetname = 'ArtEmis' #ArtEmis, Flickr30K,  VizWiz, COCO
assert datasetname == 'ArtEmis' ## Only work with ArtEmis
if datasetname == 'ArtEmis':
    datafile = f'../Dataset/{datasetname}/{datasetname}_IdC/{datasetname}_IdCII_3ErrType.csv'
    img_dir = f"../Dataset/{datasetname}/{datasetname}_IdC/Images/rawImages"
    df = pd.read_csv(datafile)
    df = df[df.split=='test']
else:
    datafile = f'../Dataset/{datasetname}/{datasetname}_IdCII_3ErrType.csv'
    img_dir = f"../Dataset/{datasetname}/Images/rawImages"
    df = pd.read_csv(datafile)
df.reset_index(drop=True,inplace=True)
print('Number of caption sets in the test set:', len(df))
df.img_files = [osp.join(img_dir,imgfile) for imgfile in df.img_files]

Number of caption sets in the test set: 15884


In [3]:
df['captSet_text'] = df['captSet_text'].apply(literal_eval)
df['refCaptSet'] = df['refCaptSet'].apply(literal_eval)
img_feat_dir = 'output/feat_ViLBERTScore' # Extracted using ExtractImgFeature_ViLBERTScore.ipynb


In [4]:
## Calculate ViLBERTscore based on https://github.com/hwanheelee1993/ViLBERTScore
import sys
import os
import torch
import yaml
import logging
logging.getLogger("pytorch_transformers").setLevel(logging.CRITICAL)
logging.getLogger("transformers").setLevel(logging.CRITICAL)
logging.getLogger("vilbert.utils").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

from easydict import EasyDict as edict
from pytorch_transformers.tokenization_bert import BertTokenizer
from ViLBERTScore.vilbert.datasets import ConceptCapLoaderTrain, ConceptCapLoaderVal
from ViLBERTScore.vilbert.vilbert import VILBertForVLTasks, BertConfig, BertForMultiModalPreTraining
from ViLBERTScore.vilbert.task_utils import LoadDatasetEval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

from PIL import Image
import cv2
import argparse
import glob
from types import SimpleNamespace
import pdb

import pickle
import json
from tqdm import tqdm
from scipy.stats import kendalltau
from torch.nn.functional import softmax
from utils import *
from ViLBERTScore.dataset import CaptioningDataset
from torch.utils.data import DataLoader
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', do_lower_case=True
)
args = SimpleNamespace(from_pretrained= "ViLBERTScore/save/multi_task_model.bin",
                       bert_model="bert-base-uncased",
                       config_file="ViLBERTScore/config/bert_base_6layer_6conect.json",
                       max_seq_length=101,
                       train_batch_size=1,
                       do_lower_case=True,
                       predict_feature=False,
                       seed=42,
                       num_workers=0,
                       baseline=False,
                       img_weight=1,
                       distributed=False,
                       objective=1,
                       visual_target=0,
                       dynamic_attention=False,
                       task_specific_tokens=True,
                       tasks='1',
                       save_name='',
                       in_memory=False,
                       batch_size=1,
                       local_rank=-1,
                       split='mteval',
                       clean_train_sets=True,
                       dataset=datasetname,
                       task=7,
                       layer=-1,
                       expname='pretrain_cls_sep'
                      )
args.task = 7
args.layer = -1
args.from_pretrained = 'ViLBERTScore/save/multi_task_model.bin'
args.compute_correlation = False

if(args.from_pretrained == 'ViLBERTScore/save/pretrained_model.bin'):
    args.task_specific_tokens = False

config = BertConfig.from_json_file(args.config_file)
with open('ViLBERTScore/vilbert_tasks.yml', 'r') as f:
    task_cfg = edict(yaml.safe_load(f))


task_names = []
for i, task_id in enumerate(args.tasks.split('-')):
    task = 'TASK' + task_id
    name = task_cfg[task]['name']
    task_names.append(name)

timeStamp = args.from_pretrained.split('/')[-1] + '-' + args.save_name
config = BertConfig.from_json_file(args.config_file)
default_gpu=True

if args.predict_feature:
    config.v_target_size = 2048
    config.predict_feature = True
else:
    config.v_target_size = 1601
    config.predict_feature = False

if args.task_specific_tokens:
    config.task_specific_tokens = True    

if args.dynamic_attention:
    config.dynamic_attention = True

config.visualization = True
num_labels = 3129

if args.baseline:
    model = BaseBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=num_labels, default_gpu=default_gpu
        )
else:
    model = VILBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=num_labels, default_gpu=default_gpu
        )
    
model.eval()
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)
from ViLBERTScore.utils import *
def compute_bert_score(model, text_a, input_mask_a, segment_ids_a, 
                       features, spatials, image_mask, co_attention_mask, input_idf_a, task, x, y, 
                       use_idf=False, layer=-1):
    with torch.no_grad():
        p_allRef = []
        r_allRef = []
        f_allRef = []            

        st_c, sv_c, pt_c, pv_c, att_c = model.bert(
            text_a[:,x,:], features, spatials, segment_ids_a[:,x,:],
            input_mask_a[:,x,:], image_mask, co_attention_mask, task,
        output_all_encoded_layers=True)
        for i in range(text_a.size(1)-y):#range(5):
            st_g, sv_g, pt_g, pv_g, att_g = model.bert(
                text_a[:,y+i,:], features, spatials, segment_ids_a[:,y+i,:],
                input_mask_a[:,y+i,:], image_mask, co_attention_mask, task,
            output_all_encoded_layers=True)
            if(use_idf):
                p, r, f = bert_score(st_g[layer], st_c[layer], input_idf_a[:,y+i, :], input_idf_a[:,x, :])
            else:
                p, r, f = bert_score(st_g[layer], st_c[layer], input_mask_a[:,y+i, :], input_mask_a[:,x, :])
            p_allRef.append(p)
            r_allRef.append(r)
            f_allRef.append(f)        

    p_allRefa = np.average(p_allRef, axis=0)
    r_allRefa = np.average(r_allRef, axis=0)
    f_allRefa = np.average(f_allRef, axis=0) 
    
    return p_allRefa, r_allRefa, f_allRefa



06/10/2022 11:05:06 - INFO - ViLBERTScore.vilbert.utils -   loading weights file ViLBERTScore/save/multi_task_model.bin


In [5]:
no_imgs =len(df)
cands = []
refs = []
imgfeatfiles = []
for _,row in df.iterrows():
    refs_ =row['refCaptSet']
    for cand in row['captSet_text']:
        cands.append(cand)
        refs.append(refs_)
        imgfeatfiles.append(osp.join(img_feat_dir, row.art_style,  row.painting + '.npy')) 
    
dataset = CaptioningDataset(cands,refs,imgfeatfiles,data_type=args.dataset, use_idf=False)
dataloader = DataLoader(dataset, 1, shuffle=False)
layer = args.layer

prs_a = []
rcs_a = []
f1s_a = []

use_idf = False

for text_a, input_mask_a, segment_ids_a, features, spatials, image_mask, co_attention_mask, input_idf_a, idxs_ in tqdm(iter(dataloader)):
    text_a = text_a.cuda() 
    input_idf_a = input_idf_a.cuda()
    input_mask_a = input_mask_a.cuda()
    segment_ids_a = segment_ids_a.cuda()
    features = features.cuda()
    spatials = spatials.cuda()
    image_mask = image_mask.cuda()
    co_attention_mask = co_attention_mask.cuda()
    task = [args.task]
    task = torch.from_numpy(np.array(task)).cuda().unsqueeze(0).repeat(spatials.size(0), 1)
    #break
    with torch.no_grad():
        p5a, r5a, f5a = compute_bert_score(model, text_a, input_mask_a, segment_ids_a, 
                                           features, spatials, image_mask, 
                                           co_attention_mask, input_idf_a, task, 
                                           0, 1, layer=layer)
        if(len(prs_a) == 0):
            prs_a = p5a
            rcs_a = r5a
            f1s_a = f5a            
        else:
            prs_a = np.concatenate((prs_a, p5a))
            rcs_a = np.concatenate((rcs_a, r5a))
            f1s_a = np.concatenate((f1s_a, f5a))

scores_P = prs_a
scores_R = rcs_a
scores_F1 = f1s_a

100%|██████████| 63536/63536 [4:07:27<00:00,  4.28it/s]  


In [6]:
scores_P = np.array(scores_P).reshape([-1,len(row['captSet_text'])])
scores_R = np.array(scores_R).reshape([-1,len(row['captSet_text'])])
scores_F1 = np.array(scores_F1).reshape([-1,len(row['captSet_text'])])

In [7]:
import numpy as np
no_errType = 3
cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",len(scores_P))
print("ViLBERTscores_P")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in scores_P:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))

cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",len(scores_R))
print("ViLBERTscores_R")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in scores_R:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))

cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",len(scores_F1))
print("ViLBERTscores_F1")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in scores_F1:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))


Dataset: ArtEmis , Number of caption sets: 15884
ViLBERTscores_P
Accuracy at errType=1:12264/15884= 0.772097708385797
Accuracy at errType=2:8955/15884= 0.5637748677914883
Accuracy at errType=3:2804/15884= 0.17652984134978594
Accuracy for all types:24023/47652= 0.5041341391756904
Dataset: ArtEmis , Number of caption sets: 15884
ViLBERTscores_R
Accuracy at errType=1:11217/15884= 0.7061823218332913
Accuracy at errType=2:10864/15884= 0.6839587005791992
Accuracy at errType=3:9130/15884= 0.574792243767313
Accuracy for all types:31211/47652= 0.6549777553932679
Dataset: ArtEmis , Number of caption sets: 15884
ViLBERTscores_F1
Accuracy at errType=1:12255/15884= 0.7715311004784688
Accuracy at errType=2:9841/15884= 0.6195542684462352
Accuracy at errType=3:4442/15884= 0.27965248048350544
Accuracy for all types:26538/47652= 0.5569126164694032
